# Q1

In [131]:
import requests
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt

#### Get data from Upbit

In [45]:
# Get Upbit Kline price
def get_upbit_data(symbol, interval, start_time, end_time):
    base_url = "https://api.upbit.com/v1/candles/minutes/60"  
    params = {
        'market': symbol,
        'to': datetime.utcfromtimestamp(end_time).strftime('%Y-%m-%dT%H:%M:%S'),
        'count': 200  
    }
    
    data = []
    while True:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        klines = response.json()
        data.extend(klines)
        if len(klines) < 200:
            break
        end_time = datetime.strptime(klines[-1]['candle_date_time_utc'], '%Y-%m-%dT%H:%M:%S').timestamp()
        params['to'] = datetime.utcfromtimestamp(end_time).strftime('%Y-%m-%dT%H:%M:%S')
    
    df = pd.DataFrame(data)
    df['timestamp'] = pd.to_datetime(df['candle_date_time_utc'])
    df.set_index('timestamp', inplace=True)
    df = df[['opening_price', 'high_price', 'low_price', 'trade_price', 'candle_acc_trade_volume']]
    df.columns = ['open', 'high', 'low', 'close', 'volume']
    df = df.astype(float)
    
    return df

start_time = int(datetime(2021, 1, 1).timestamp())
end_time = int(datetime(2023, 1, 1).timestamp())

upbit_data = get_upbit_data('KRW-BTC', '60', start_time, end_time)

upbit_data.head()

                           open        high         low       close  \
timestamp                                                             
2022-12-31 15:00:00  21164000.0  21225000.0  21100000.0  21127000.0   
2022-12-31 14:00:00  21226000.0  21248000.0  21133000.0  21164000.0   
2022-12-31 13:00:00  21155000.0  21249000.0  21151000.0  21223000.0   
2022-12-31 12:00:00  21131000.0  21169000.0  21080000.0  21159000.0   
2022-12-31 11:00:00  21067000.0  21136000.0  21067000.0  21124000.0   

                         volume  
timestamp                        
2022-12-31 15:00:00  256.574817  
2022-12-31 14:00:00  343.777366  
2022-12-31 13:00:00  739.587108  
2022-12-31 12:00:00  224.296491  
2022-12-31 11:00:00  299.865181  


In [47]:
upbit_close_data = upbit_data[['close']]
upbit_close_data

close
timestamp                      
2022-12-31 15:00:00  21127000.0
2022-12-31 14:00:00  21164000.0
2022-12-31 13:00:00  21223000.0
2022-12-31 12:00:00  21159000.0
2022-12-31 11:00:00  21124000.0
...                         ...
2017-09-25 11:00:00   4191000.0
2017-09-25 10:00:00   4204000.0
2017-09-25 08:00:00   4235000.0
2017-09-25 04:00:00   4235000.0
2017-09-25 03:00:00   4202000.0

[44325 rows x 1 columns]

#### Get data from Binance

In [46]:
# Get binance Kline price
def get_binance_data(symbol, interval, start, end):
    base_url = "https://fapi.binance.com/fapi/v1/klines"
    params = {
        'symbol': symbol,
        'interval': interval,
        'startTime': start,
        'endTime': end,
        'limit': 1000
    }
    
    data = []
    while True:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        klines = response.json()
        data.extend(klines)
        if len(klines) < 1000:
            break
        params['startTime'] = klines[-1][0] + 1
    
    df = pd.DataFrame(data, columns=[
        'timestamp', 'open', 'high', 'low', 'close', 'volume', 
        'close_time', 'quote_asset_volume', 'number_of_trades', 
        'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
    ])
    
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df = df.astype(float)
    
    return df

start = int(datetime(2021, 1, 1).timestamp() * 1000)
end = int(datetime(2023, 1, 1).timestamp() * 1000)

binance_data = get_binance_data('BTCUSDT', '1h', start, end)

binance_data.head()

                         open      high       low     close     volume  \
timestamp                                                                
2020-12-31 16:00:00  28792.58  28837.77  28327.78  28408.72  13071.080   
2020-12-31 17:00:00  28407.73  28750.00  28378.10  28603.10   9192.806   
2020-12-31 18:00:00  28601.52  28923.71  28501.00  28883.17   8127.876   
2020-12-31 19:00:00  28883.17  29037.09  28761.52  28917.08   8298.073   
2020-12-31 20:00:00  28917.08  29180.99  28888.00  29155.97   5985.123   

                       close_time  quote_asset_volume  number_of_trades  \
timestamp                                                                 
2020-12-31 16:00:00  1.609434e+12        3.735963e+08           91955.0   
2020-12-31 17:00:00  1.609438e+12        2.628727e+08           70382.0   
2020-12-31 18:00:00  1.609441e+12        2.336889e+08           64817.0   
2020-12-31 19:00:00  1.609445e+12        2.399987e+08           65281.0   
2020-12-31 20:00:00  1.609448e+

In [49]:
binance_close_data = binance_data[['close']]
binance_close_data

close
timestamp                    
2020-12-31 16:00:00  28408.72
2020-12-31 17:00:00  28603.10
2020-12-31 18:00:00  28883.17
2020-12-31 19:00:00  28917.08
2020-12-31 20:00:00  29155.97
...                       ...
2022-12-31 12:00:00  16576.10
2022-12-31 13:00:00  16597.70
2022-12-31 14:00:00  16584.80
2022-12-31 15:00:00  16586.00
2022-12-31 16:00:00  16596.00

[17521 rows x 1 columns]

#### Merge the data

In [132]:
upbit_close_data = upbit_close_data.sort_values('timestamp')
binance_close_data = binance_close_data.sort_values('timestamp')
merged_close_data = pd.merge_asof(upbit_close_data,binance_close_data,on='timestamp')
merged_close_data.set_index('timestamp', inplace=True)
merged_close_data.columns = ['Upbit','Binance']
merged_close_data = merged_close_data.dropna(subset=['Binance'])

merged_close_data

Upbit   Binance
timestamp                                
2020-12-31 16:00:00  31679000.0  28408.72
2020-12-31 17:00:00  31767000.0  28603.10
2020-12-31 18:00:00  31999000.0  28883.17
2020-12-31 19:00:00  32014000.0  28917.08
2020-12-31 20:00:00  32200000.0  29155.97
...                         ...       ...
2022-12-31 11:00:00  21124000.0  16565.60
2022-12-31 12:00:00  21159000.0  16576.10
2022-12-31 13:00:00  21223000.0  16597.70
2022-12-31 14:00:00  21164000.0  16584.80
2022-12-31 15:00:00  21127000.0  16586.00

[16824 rows x 2 columns]

#### The strategy

In [202]:
def strategy(df, x, y):
    signals = []
    df_copy = df.copy()
    df_copy.loc[:,'Upbit_price_change_pct'] = df_copy['Upbit'].pct_change() * 100
    for i in range(1,len(df_copy)):
        price_change_pct = df_copy.iloc[i,2]
        
        if price_change_pct >= x:
            signals.append(1)
        elif price_change_pct <= -y:
            signals.append(-1)
        else:
            signals.append(0)

    return signals

#### Split the data

In [203]:
split_date = merged_close_data.index[int(len(merged_close_data) / 2)+1]
backtest_data = merged_close_data[:split_date]

backtest_data.head()

Upbit   Binance
timestamp                                
2020-12-31 16:00:00  31679000.0  28408.72
2020-12-31 17:00:00  31767000.0  28603.10
2020-12-31 18:00:00  31999000.0  28883.17
2020-12-31 19:00:00  32014000.0  28917.08
2020-12-31 20:00:00  32200000.0  29155.97

In [204]:
forward_data = merged_close_data[split_date:]
forward_data.head()

Upbit   Binance
timestamp                                
2021-12-31 23:00:00  56784000.0  46210.56
2022-01-01 00:00:00  57053000.0  46650.01
2022-01-01 01:00:00  57181000.0  46766.78
2022-01-01 02:00:00  57258000.0  46796.14
2022-01-01 03:00:00  57276000.0  46789.56

#### Initialize the investment and calculate CAGR, Maximum Drawdown and Sharpe Ratio

In [205]:
def metric(returns):
    CAGR = returns[-1]
    MaxDraw = min(returns)
    mean = np.mean(returns)
    std = np.std(returns,ddof = 1)
    sharpe_ratio = mean / std

    return CAGR, MaxDraw, sharpe_ratio


def cum_return(merged_close_data, signals):
    binance = merged_close_data['Binance']
    bitcoin_postions = [0]
    balances = [500000] # the initial balance is $500,000
    return_list = [0]
    for i in range(0,len(signals)):
        bitcoin_postions.append(bitcoin_postions[i]+signals[i])
        balances.append(balances[i] + bitcoin_postions[i] * binance[i+1] - bitcoin_postions[i] * binance[i])
        return_list.append(balances[i] / balances[0] - 1)
        
    return return_list

#### Draw the sharpe ratio heat map

In [206]:
def generate_sharpe_heatmap(df):
    x_values = np.arange(0, 8, 0.5)
    y_values = np.arange(2, 10, 0.5)
    sharpe_ratios = np.zeros((len(x_values), len(y_values)))
    
    for i, x in enumerate(x_values):
        for j, y in enumerate(y_values):
            df_copy = df.copy()
            signals = strategy(df_copy, x, y)
            return_list = cum_return(df_copy, signals)
            _,_,sharpe_ratio = metric(return_list)
            sharpe_ratios[i, j] = sharpe_ratio
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(sharpe_ratios, xticklabels=x_values, yticklabels=y_values, annot=True, cmap="crest")
    plt.title('Sharpe Ratio Heatmap')
    plt.xlabel('X%')
    plt.ylabel('Y%')
    plt.show()

In [200]:
generate_sharpe_heatmap(backtest_data)

So, it can be seen that the best (x,y) sets are (5.0 ~ 7.5,9.5), and the maximum sharpe ratio is 1.2. And according to the change of the sharpe ratio, the best
(x, y) set may be (7.5, 9.5).

#### Calculate CAGR, Maximum Drawdown and Sharpe Ratio in backtest dataset and forward dataset.

In [224]:
# Backtest data
signals_back = strategy(backtest_data, 7.5, 9.5)
return_list = cum_return(backtest_data, signals_back)
CAGR,Max_draw,sharpe_ratio = metric(return_list)
print(f'The CAGR equals to: {CAGR}, the Maximum Drawdown is: {Max_draw}, and the sharpe ratio is: {sharpe_ratio}')

The CAGR equals to: 0.019800599999986623, the Maximum Drawdown is: -0.014322980000010088, and the sharpe ratio is: 1.1538145225096579


In [226]:
# Forward data
signals_forward = strategy(forward_data, 7.5, 9.5)
return_list = cum_return(forward_data, signals_forward)
CAGR,Max_draw,sharpe_ratio = metric(return_list)
print(f'The CAGR equals to: {CAGR}, the Maximum Drawdown is: {Max_draw}, and the sharpe ratio is: {sharpe_ratio}')

The CAGR equals to: 0.0, the Maximum Drawdown is: 0, and the sharpe ratio is: nan


C:\Users\calin\AppData\Local\Temp\ipykernel_5572\2064845456.py:6: RuntimeWarning: invalid value encountered in double_scalars
  sharpe_ratio = mean / std


It can be found that (7.5, 9.5) can not have a sharpe ratio in forward test dataset, that may because the backtest dataset is too specific on a particular time, and (x,y) are too extreme. If we change the (x,y) set to (7.5, 4.5). The maximum sharpe ratio for the backtest dataset becomes 0.66. And CAGR, Maximum Drawdown and Sharpe Ratio in forward dataset are shown as follow:

In [233]:
# Forward data
signals_forward = strategy(forward_data, 7.5, 4.5)
return_list = cum_return(forward_data, signals_forward)
CAGR,Max_draw,sharpe_ratio = metric(return_list)
print(f'The CAGR equals to: {CAGR}, the Maximum Drawdown is: {Max_draw}, and the sharpe ratio is: {sharpe_ratio}')

The CAGR equals to: 0.03448239999999991, the Maximum Drawdown is: -0.005576000000000025, and the sharpe ratio is: 0.9764380943255269
